In [587]:
import json, hashlib, requests, json, time

In [588]:
# Marvel API data
endpoint = "https://gateway.marvel.com/v1/public/"
apiKey = "6818695382c18243570eb27db6d83eb3"
privateKey = "0ca11a0a359ac649d46d934064349266dde56d3d"
limit = "100"

In [589]:
comics = open('comics.json', 'r')
comics_data = json.load(comics)
comics.close()

In [590]:
started = time.time()
print("Started at: " + str(started))

Started at: 1517043663.49079


In [591]:
skipped_comics = [] # Array with id of comics which have more than 100 characters
_cbc = []

# Cycle through the database
for comic in range(0, len(comics_data)):
    
    # Generate md5 hash from both api keys and timestamp
    timestamp = str(int(time.time()))
    hs = hashlib.md5(timestamp.encode('utf-8') + privateKey.encode('utf-8') + apiKey.encode('utf-8')).hexdigest()
    
    # Make the request
    response = requests.get(endpoint + 'comics/' + str(comics_data[comic]['id']) + '/characters'
                            + "?" + "orderBy=name"
                            + "&" + "limit=" + limit
                            + "&" + "ts=" + timestamp 
                            + "&" + "apikey=" + apiKey
                            + "&" + "hash=" + hs)
    
    # Break if at least 1 
    # request was unsuccessful
    if(response.status_code != 200):
        print("An error " + str(response.status_code) + " occured!")
        break
    
    # Extract the data about characters
    data = response.json()['data']
    
    if data['count'] < data['total']:
        skipped_comics.append(comic['id'])
        continue
    
    _comic = {}
    _comic['comic_id'] = comics_data[comic]['id']

    results = data['results']
    characters_ids = []
    for character in results:
        characters_ids.append(character['id'])
    _comic['characters'] = characters_ids
    
    _cbc.append(_comic)
    
    # Sleep for some time
    if ((comic % 50) == 0):
        time.sleep(1)

In [592]:
finished = time.time()
print("Time elapsed: " + str((finished - started) / 60))

Time elapsed: 5.83885733683904


In [593]:
cbc = open('characters_by_comics.json', 'w+')

# Clear the file to update it
cbc.seek(0)
cbc.truncate()

# Write to the file
json.dump(_cbc, cbc, indent=2)

cbc.close()

In [596]:
if(len(skipped_comics) > 0):
    print(skipped_comics)
else:
    print("No comics were skipped")

No comics were skipped
